# Classification of Iris Data Set

## Import Libraries

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Loading the Iris Data Set

In [4]:
iris = load_iris()
#print(iris)

## Now we will process the Iris Data Set

In [5]:
features = iris.data
#print(features)
target = iris.target
#print(len(target))

## Split the Data Set

In [ ]:
#splitting to 20% test 80% train
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

## Now we will standardize the values 

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Initiaing the number of classes and features for the training function

In [ ]:
features_count = x_train.shape[1]
print(features_count)
classes = len(np.unique(target))
print(classes)

## Converting them into Pytorch Tensors

In [ ]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

## Building up the model 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dnn_model = Sequential(Linear(features_count,10), nn.ReLU(),
                       Linear(10,20), nn.ReLU(),
                       Linear(20,15), nn.ReLU(),
                       Linear(15, classes)).to(device)

# Computing accuracy

In [ ]:
def compute_acc(model, X_data, y_data, device):
    model.eval() #Evaluation Model
    with torch.no_grad(): #Disable gradient calculation to save computation energy 
        X_data, y_data = X_data.to(device), y_data.to(device)
        #use the model to predict the results
        outputs = model(X_data.type(torch.float32))  
        #choose the highest probability for each row so it belongs to the certain class
        predicted = torch.argmax(outputs.data, 1)
        corrects = (predicted == y_data.type(torch.long)).sum().item()
        totals = y_data.size(0)
        acc = float(corrects) / totals
    return acc

## Function to create model 

In [ ]:
def create_model():
	dnn_model = Sequential(Linear(features_count,10), nn.ReLU(),
                         Linear(10,20), nn.ReLU(),
                         Linear(20,15), nn.ReLU(),
                         Linear(15, classes))
	return dnn_model

## Training model 

In [ ]:
def fit(model=None, X_train=None, y_train=None, loss_fn=None, optimizer=torch.optim.Adam,
        learning_rate=0.001, num_epochs=100, verbose=True, seed=1234, device=None):
    torch.manual_seed(seed)
    optim = optimizer(model.parameters(), lr=learning_rate)
    history = dict()
    history['train_loss'] = []
    history['train_acc'] = []

    # Move data to device
    X_train, y_train = X_train.to(device), y_train.to(device)

    for epoch in range(num_epochs):
        #allow the model to go in training mode
        model.train()
        
        #foward propaogation
        outputs = model(X_train.type(torch.float32))
        loss = loss_fn(outputs, y_train.type(torch.long))
        
        #make sure the gradient computed is zero so it doesnt accumulate from previous iteration
        optim.zero_grad()
        #conpute the gradient lost
        loss.backward()
        #update the weights
        optim.step()

        #evaluate the accuracy for this current epoch
        model.eval()  # Set the model to evaluation mode
        train_loss = compute_loss(model, loss_fn, X_train, y_train, device)
        train_acc = compute_acc(model, X_train, y_train, device)

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        if verbose:
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"train loss= {train_loss:.4f} - train acc= {train_acc*100:.2f}%")

    return history

## Training model and choosing the optimiser 

In [ ]:
from torch import optim
optim_dict = {"Adam":optim.Adam, "Adadelta":optim.Adadelta, "Adagrad":optim.Adagrad,
              "Adamax":optim.Adamax, "AdamW": optim.AdamW, "ASGD":optim.ASGD,
              "NAdam":optim.NAdam, "RMSprop":optim.RMSprop, "RAdam":optim.RAdam,
              "Rprop": optim.Rprop, "SGD":optim.SGD}

dnn_model = create_model().to(device)
history = fit(dnn_model, X_train = x_train_tensor, y_train=y_train_tensor, loss_fn = nn.CrossEntropyLoss(),
    optimizer = optim_dict["SGD"], learning_rate = 0.1, num_epochs = 50, verbose= True, seed=123, device=device)

## Testing the model  

In [ ]:
test_accuracy = compute_acc(dnn_model, x_test_tensor, y_test_tensor, device)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")